In [ ]:
#Prerequisites

import sys
!{sys.executable} -m pip install --user networkx
!{sys.executable} -m pip install --user numpy
!{sys.executable} -m pip install --user pandas

import networkx as nx
import numpy as np
import pandas as pd
import os

from pandas import DataFrame

In [ ]:
# Read in Source File - NB this must match the schema requirements
# read in data
initial_report_SF = pd.read_csv("~/Documents/Trellis/Network_Analysis_Tableau/data/initial-report-SF2.csv")
CodeType = 'latin1' # https://docs.python.org/3/library/codecs.html#standard-encodings
print(initial_report_SF.tail())

In [ ]:
# Get variable names (column names) of the DataFrame
variable_names = initial_report_SF.columns

print(variable_names)

In [ ]:
# Filter out rows where `Case ID` or `Contact EmplID` is NA or empty
filtered_df = initial_report_SF.dropna(subset=['Case Number', 'Case ID', 'Contact EmplID'])
#initial_report_SF = initial_report_SF[(~initial_report_SF['Case ID'].isna()) | (initial_report_SF['Case ID'] != "")]
#initial_report_SF = initial_report_SF[(~initial_report_SF['Contact EmplID'].isna()) | (initial_report_SF['Contact EmplID'] != "")]
print(filtered_df.head())

In [ ]:
# Convert 'Appointment Date' column to datetime
filtered_df['AppointmentDate'] = pd.to_datetime(filtered_df['Appointment Date'])

# Calculate today's date
today = pd.to_datetime('today').normalize()
filtered_df['today'] = today

# Calculate days since 'AppointmentDate'
filtered_df['days_since'] = (filtered_df['today'] - filtered_df['AppointmentDate']).dt.days

# filter dates to 2024
#filtered_df[(filtered_df['date']>datetime.date(2024,1,1)) & (filtered_df['date']<datetime.date(2024,5,5))]
filtered_df = filtered_df[(filtered_df['AppointmentDate'] > "2024-01-01") & (filtered_df['AppointmentDate'] < "2024-07-01")]
# Display the updated DataFrame
print(filtered_df.head())

In [ ]:
# create new emplID's
# Extract unique IDs
unique_ids = filtered_df['Contact EmplID'].unique()

# Generate new IDs (random IDs)
np.random.seed(53013)  # for reproducibility
new_ids = np.random.randint(1000, 9999, len(unique_ids))

# Create a dictionary to map old IDs to new IDs
id_mapping = dict(zip(unique_ids, new_ids))

id_mapping
#filtered_df_newID=filtered_df.append(id_mapping,ignore_index=True)


In [ ]:
# Get variable names (column names) of the DataFrame
variable_names = filtered_df.columns

print(variable_names)

In [ ]:
#filtered_df['Contact EmplID'].map(id_mapping)  
filtered_df['New_ID'] = filtered_df['Contact EmplID'].map(id_mapping)

In [ ]:
print(filtered_df.head())

In [ ]:
# create a new column to extract the names
# Extract just the names
filtered_df['Student Name'] = filtered_df['Appointment Name'].str.split(' - ', expand=True)[1]

# Display the updated DataFrame
print(filtered_df.head())

In [ ]:
# add an index column

filtered_df['index'] = range(1, len(filtered_df) + 1)
#filtered_df['index'] = filtered_df.index

# Display the updated DataFrame
#print(filtered_df.head())

In [ ]:
# create a source column
filtered_df['Source'] = filtered_df.groupby(['Advisor Name']).ngroup()

# Display the updated DataFrame
print(filtered_df.head())

In [ ]:
filtered_df = filtered_df.rename(columns={'Owner: Owner ID': 'owner_id', 'New_ID': 'Target'})

In [ ]:
filtered_df.info()

In [ ]:
filtered_df['Source'] = filtered_df['Source'].map(str)
filtered_df['Target'] = filtered_df['Target'].map(str)

In [ ]:
# Add directions to d3
filtered_df['Direction'] = filtered_df['Source'] + ' -> ' + filtered_df['Target']

In [ ]:
filtered_df.info()

In [ ]:
# FOLLOWING KNIPPENBERG

# Save the DataFrame to a CSV file

filtered_df.to_csv('~/Documents/Trellis/Network_Analysis_Tableau/data/generated_from_knipp/py-filtered_df.csv', index=False)  # Set index=False to exclude the index from the CSV file


In [ ]:
reduced_df = filtered_df.sample(n=300)

In [ ]:
arr_SrcTgt = np.array(reduced_df[['Source','Target']])

In [ ]:
# Create Network Graph Coordinates...
Q = nx.Graph()
Q.add_edges_from(arr_SrcTgt)
dict_Coords = nx.spring_layout(Q) 

In [ ]:
# Create Graph Coordinates File...
df_Raw_Coords = DataFrame(dict_Coords)
df_Raw_Coords = df_Raw_Coords.T
df_Raw_Coords.columns = ['X','Y']
df_Raw_Coords.to_csv('~/Documents/Trellis/Network_Analysis_Tableau/data/generated_from_knipp/CoordsFile.csv', index_label='NodeName')
print(df_Raw_Coords)

In [ ]:
# Create Bridge File... 
# Tableau Code: IF [Src-Tgt]/2 = ROUND([Src-Tgt]/2) THEN 'Source' ELSE 'Target' END
arr_SrcTgt2 = arr_SrcTgt.reshape(1,(len(arr_SrcTgt)*2)) 
arr_SrcTgt2 = arr_SrcTgt2.reshape(-1) 
df_SrcTgt = DataFrame(arr_SrcTgt2,columns=['NodeName']) 
arr_Index = []
for i in range(1,(len(arr_SrcTgt)+1)):
    arr_Index.append(i)
    arr_Index.append(i)
df_SrcTgt['c_Index'] = arr_Index 

df_SrcTgt.to_csv('~/Documents/Trellis/Network_Analysis_Tableau/data/generated_from_knipp/BridgeFile.csv',index_label='Src-Tgt')
#print(df_ScrTgt.head())

In [ ]:
print('Run Completed Successfully')